Юхимюк Роман

In [8]:
import gensim.models as gm
import pandas as pd
import string
import spacy
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial

ПІДГОТОВКА

In [2]:
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
stemmer = SnowballStemmer(language='english')

In [3]:
#=================================================
# 1
def delete_non_literal(txt):
    ascii = (string.ascii_letters + " ")
    data1 = (c for c in txt if c in ascii)
    txt1 = "".join(data1)
    return ' '.join(txt1.split())


# 2
def tokenization(txt):
    return [str(tok) for tok in txt]


# 3
def delete_stopwords(tokenn):
    return [tok for tok in tokenn if not tok in spacy_stopwords]


# 4
def case_change(tokenn):
    return [tok.lower() for tok in tokenn]


# 5
def stemming(tokenn):
    return [stemmer.stem(tok) for tok in tokenn]


# 6
def lemmatization(txt):
    return [tok.lemma_ for tok in txt]


def all(txt):
    x = nlp(delete_non_literal(txt))
    a = tokenization(x)
    b = delete_stopwords(case_change(a))
    c = lemmatization(nlp(' '.join(b)))
    return stemming(c)
#===================================

Знайти текстовий датасет для задач ранкінгу.
Датасет текстів:

In [5]:
df = pd.read_csv('APPLE_iPhone_SE.csv', encoding='utf8')
df = df[["Ratings", "Reviews"]].dropna()
df

,Ratings,Reviews
0,5,Great camera for pics and videos Battery life ...
1,5,Great device. Let me tell the Pros..1. Superb ...
2,5,"Who all loves older size i.e., 4.7 inch type s..."
3,5,This iPhone SE is the best phone ever you get....
4,5,This is my second iphone after iphone 4s. I’ve...
...,...,...
9708,5,Absolutely brilliantREAD MORE
9709,5,"Superb phone. This is my 4th iPhone, I feel SE..."
9710,5,very niceREAD MORE
9711,5,Loving it as of now. Good Product .READ MORE


In [6]:
df2 = df["Reviews"]

II. Підготувати дані до обробки

Підготовка даних:

In [7]:
arr = []
for i in df2:
    lst = all(i)
    arr.append(lst)

In [9]:
arr1 = [' '.join(i) for i in arr]

III. Побудувати tf-idf модель

IV. Перетворити тексти на вектори

In [10]:
tf_idf = TfidfVectorizer()
X = tf_idf.fit_transform(arr1)
X = X.toarray()

V. Сформувати пошуковий запит і також його векторизувати

In [11]:
request = "Cool phone. Waterproof, can be drowned. The camera is cool and the design is beautiful. The powerful battery will surprise you, and the price is quite attractive."
req = ' '.join(all(request))

In [12]:
reqvec = tf_idf.transform([req])
reqvec = reqvec.toarray()[0]

VI. Знайти найближчі до вектору запиту 10 векторів текстів за допомогою
косинусної відстані. Для цього розрахувати косинусну відстань між векторами
усіх текстів та вектором запиту.

In [13]:
arr_simmilarity = [spatial.distance.cosine(i, reqvec) for i in X]

In [14]:
a = (np.argsort(arr_simmilarity))[:10]

array([5184, 4611, 2405, 3659,  442, 5374, 9168,  537, 4339, 7960])

In [18]:
texts = [df2.iloc[i] for i in a]

In [19]:
texts

['Cool oneREAD MORE',
 'Very cool product I am loving itREAD MORE',
 'The device looks good and attractive for the price. Smart, Handy and cool mobile to have.READ MORE',
 'I’m happy with this cool iPhone seREAD MORE',
 'Awesome product... Super fast processor.... camera is good... cool design... Red is really nice...READ MORE',
 'Cool very fast and best in classREAD MORE',
 'Cool product with 2 sim from iPhone. Loved it.READ MORE',
 'Best 1 with a cool design and development.good processing as well as gud camera.READ MORE',
 'Super cool Phone u you have take 128gb romREAD MORE',
 'Really an amazing deal at 25 K. And the color red is so cool. Love itREAD MORE']

Корпус має зчитуватись із файлів. Запит та найближчі до нього тексти мають
бути виведені в файл output.txt. Примітка – у файл вивести саме початкові
тексти, а не їх вектори!

In [22]:
with open('output.txt', 'w', encoding='utf-8') as fw:
    fw.write("Request: " + request + "\n")
    fw.write("Similar texts: \n")
    for i in texts:
          fw.write(i)
          fw.write("\n")